In [11]:
import os
import random
import json
import math

import numpy as np
import torch as th
import torch.nn as nn
from tqdm import tqdm

In [3]:
dur_file="/disk/scratch_fast/s2004019/youcook2/yc2/yc2_duration_frame.csv"
annotation_file="/disk/scratch_fast/s2004019/youcook2/yc2/yc2_new_annotations_trainval_test.json"

In [71]:
dur_frame_dict = {}
with open(dur_file, "r") as f:
    for line in f:
        vid_name, vid_dur, vid_frame = [
            l.strip() for l in line.split(',')]
        dur_frame_dict[vid_name] = (float(vid_dur), int(vid_frame))
    
with open(annotation_file, "r") as f:
    my_master = json.load(f)

In [6]:
full_annotation_file="/disk/scratch_fast/s2004019/youcook2/yc2/yc2_annotations_trainval.json"
with open(full_annotation_file, "r") as f:
    master_full = json.load(f)

In [7]:
full_validation = {}
for key, item in master_full["database"].items():
    if item["subset"] == "validation":
        full_validation[key] = item

len(full_validation)

457

In [186]:
my_validation = {}
for key, item in my_master["database"].items():
    if item["subset"] == "validation":
        my_validation[key] = item

print(len(my_validation))


my_testing = {}
for key, item in my_master["database"].items():
    if item["subset"] == "testing":
        my_testing[key] = item

len(my_testing)

183


180

In [24]:
segment_duration_full = {k:0 for k in range(200)}
for vid, ann in full_validation.items():
    for segment in ann["annotations"]:
        length = math.ceil(segment["segment"][1] - segment["segment"][0])
        segment_duration_full[length] += 1

In [96]:
print(segment_duration_full)

{0: 0, 1: 14, 2: 67, 3: 101, 4: 152, 5: 159, 6: 138, 7: 174, 8: 139, 9: 175, 10: 170, 11: 122, 12: 116, 13: 131, 14: 126, 15: 122, 16: 96, 17: 81, 18: 90, 19: 77, 20: 91, 21: 73, 22: 74, 23: 72, 24: 64, 25: 54, 26: 47, 27: 37, 28: 46, 29: 38, 30: 52, 31: 31, 32: 26, 33: 26, 34: 20, 35: 26, 36: 22, 37: 28, 38: 26, 39: 23, 40: 20, 41: 13, 42: 16, 43: 15, 44: 16, 45: 14, 46: 13, 47: 6, 48: 14, 49: 13, 50: 16, 51: 13, 52: 15, 53: 10, 54: 8, 55: 11, 56: 8, 57: 3, 58: 13, 59: 2, 60: 6, 61: 3, 62: 6, 63: 5, 64: 3, 65: 4, 66: 5, 67: 5, 68: 3, 69: 4, 70: 3, 71: 3, 72: 2, 73: 5, 74: 1, 75: 1, 76: 4, 77: 5, 78: 3, 79: 0, 80: 4, 81: 1, 82: 2, 83: 2, 84: 2, 85: 2, 86: 2, 87: 0, 88: 2, 89: 2, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 2, 96: 0, 97: 0, 98: 0, 99: 1, 100: 1, 101: 1, 102: 0, 103: 0, 104: 2, 105: 0, 106: 1, 107: 0, 108: 1, 109: 1, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 1, 120: 0, 121: 1, 122: 1, 123: 0, 124: 0, 125: 1, 126: 0, 127: 1, 128: 0, 129: 0, 13

In [97]:
x = {k: v for k, v in sorted(segment_duration_full.items(), key=lambda item: item[1]) if v > 100}

In [99]:
print(np.median(list(x.values())))
print(x)

138.0
{3: 101, 12: 116, 11: 122, 15: 122, 14: 126, 13: 131, 6: 138, 8: 139, 4: 152, 5: 159, 10: 170, 7: 174, 9: 175}


In [56]:
print(x.values()), print(np.median(list(x.values())))

dict_values([11, 13, 13, 13, 13, 13, 14, 14, 14, 15, 15, 16, 16, 16, 20, 20, 22, 23, 26, 26, 26, 26, 28, 31, 37, 38, 46, 47, 52, 54, 64, 67, 72, 73, 74, 77, 81, 90, 91, 96, 101, 116, 122, 122, 126, 131, 138, 139, 152, 159, 170, 174, 175])
46.0


(None, None)

In [49]:
x1 = np.array(list(x.values()))

In [53]:
len(x1[x1==3])

7

In [121]:
def get_uniform_label_withones(segments, total, max_len=30):
    ones_in_segment = math.ceil(total / (2*segments))
    ones_in_segment = min(ones_in_segment, max_len)
    total_zeros = total - ones_in_segment * segments
    one_segment_label = [1]*ones_in_segment
    segment_pre_zero = math.floor(total_zeros / segments)
    zero_segment_label = [0]*segment_pre_zero
    label = []
    for k in range(segments):
        label.append(zero_segment_label)
        label.append(one_segment_label)
    ll = [item for sublist in label[:total] for item in sublist]
    return ll

In [130]:
def get_labels_start_end_time(frame_wise_labels, bg_class=["0"]):
    labels = []
    starts = []
    ends = []
    last_label = frame_wise_labels[0]
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i-1)
            last_label = frame_wise_labels[i]
    if last_label not in bg_class:
        ends.append(i)
    return labels, starts, ends


def mean_iou(recognized, ground_truth, overlap, bg_class=["background"]):
    p_label, p_start, p_end = get_labels_start_end_time(recognized, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(ground_truth, bg_class)
    ious = []
    hits = np.zeros(len(y_label))
    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        intersection[intersection <= 0] = 0
        union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
        IoU = (1.0*intersection / union)*([p_label[j] == y_label[x] for x in range(len(y_label))])
        # Get the best scoring segment
        idx = np.array(IoU).argmax()
        #if not hits[idx]:
        #    hits[idx] = 1
        ious.append(IoU[idx])

    return np.mean(ious)


def mean_jaccard(recognized, ground_truth, overlap, bg_class=["background"]):
    p_label, p_start, p_end = get_labels_start_end_time(recognized, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(ground_truth, bg_class)
    jaccs = []
    hits = np.zeros(len(y_label))
    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        intersection[intersection <= 0] = 0
        denom = p_end[j] - p_start[j]
        jacc = (1.0*intersection / denom)*([p_label[j] == y_label[x] for x in range(len(y_label))])
        # Get the best scoring segment
        idx = np.array(jacc).argmax()
        # if not hits[idx]:
        #    hits[idx] = 1
        jaccs.append(jacc[idx])
    return np.mean(jaccs)

def getUniformScore(target, num_segments, bg_class=["0"], max_len=30):
    y_label, y_start, y_end = get_labels_start_end_time(target, bg_class) 
    uniform_pred = get_uniform_label_withones(len(y_label), len(target), max_len)
    recognition = []
    for i in range(len(uniform_pred)):
        recognition.append(str(int(uniform_pred[i])))
    return mean_iou(recognition, target, None, bg_class=["0"]), mean_jaccard(recognition, target, None, bg_class=["0"])

In [131]:
frames_per_video = 500
aug_number = 1

ious = []
jaccs = []
for vid, item in full_validation.items():
    annotations = item["annotations"]
    
    duration_t = dur_frame_dict[vid][0]
    total_frame = dur_frame_dict[vid][1]
    sampling_itv = math.ceil(total_frame/frames_per_video)
    time_per_sampled_frame = sampling_itv*duration_t/total_frame
    aug_frame_shift = max(math.floor(
        sampling_itv/10), 1)*frames_per_video/total_frame

    label = np.zeros(frames_per_video)
    num_segments = len(annotations)
    for ann_i, ann in enumerate(annotations):
        start = float(ann["segment"][0])
        end = float(ann["segment"][1])
        start = start/time_per_sampled_frame + \
            1-aug_frame_shift*(aug_number-1)
        end = end/time_per_sampled_frame+1-aug_frame_shift*(aug_number-1)
        label[math.floor(start):math.ceil(end)] = 1
    
    target = []
    for i in range(len(label)):
        target.append(str(int(label[i])))
    iou, jacc = getUniformScore(target, num_segments)
    ious.append(iou)
    jaccs.append(jacc)
print(np.nanmean(ious), np.nanmean(jaccs))

0.2299455494084134 0.44223830632299516


In [132]:
ious = []
jaccs = []
for vid, item in full_validation.items():
    annotations = item["annotations"]
    
    duration_t = dur_frame_dict[vid][0]
    total_frame = dur_frame_dict[vid][1]
    frame_to_second = float(duration_t)*int(float(total_frame)*1./int(float(total_frame)))*1./float(total_frame)
    fps = total_frame / duration_t
    label = np.zeros(total_frame)
    for ann_i, ann in enumerate(annotations):
        start = float(ann["segment"][0])
        end = float(ann["segment"][1])
        start = start * fps
        end = end * fps
        label[math.floor(start):math.ceil(end)] = 1
    
    target = []
    for i in range(len(label)):
        target.append(str(int(label[i])))
    iou, jacc = getUniformScore(target, len(annotations), max_len=25*25)
    ious.append(iou)
    jaccs.append(jacc)
print(np.nanmean(ious), np.nanmean(jaccs))

0.258332296109148 0.4406784675286247


In [164]:
def compute_iou(c1, c2):
    intersection = max(0, min(c1[1], c2[1])-max(c1[0], c2[0]))
    if intersection == 0:
        return 0
    else:
        union = max(c1[1], c2[1]) - min(c1[0], c2[0])
        return intersection/union

def compute_jacc(c1, c2):
    intersection = max(0, min(c1[1], c2[1])-max(c1[0], c2[0]))
    if intersection == 0:
        return 0
    else:
        union = (c1[1]-c1[0])
        return intersection/union

In [184]:
ious = []
jaccs = []
for vid, item in my_testing.items():
    annotations = item["annotations"]
    
    duration_t = dur_frame_dict[vid][0]
    total_frame = dur_frame_dict[vid][1]
    sampling_itv = math.ceil(total_frame/frames_per_video)
    time_per_sampled_frame = sampling_itv*duration_t/total_frame
    aug_frame_shift = max(math.floor(
        sampling_itv/10), 1)*frames_per_video/total_frame

    label = []
    num_segments = len(annotations)
    for ann_i, ann in enumerate(annotations):
        start = float(ann["segment"][0])
        end = float(ann["segment"][1])
#         start = start/time_per_sampled_frame + \
#             1-aug_frame_shift*(aug_number-1)
#         end = end/time_per_sampled_frame+1-aug_frame_shift*(aug_number-1)
        label.append([round(start, 5), round(end,5)])
    
    total_segs = 2 * num_segments + 1
    total_segs = num_segments
    interval = duration_t / total_segs
    
    uniform_preds = []
    for i in range(total_segs):
        #if i % 2 == 0:
        #    continue
        start = i*interval
        end = (i+1)*interval
#         start = start/time_per_sampled_frame + \
#             1-aug_frame_shift*(aug_number-1)
#         end = end/time_per_sampled_frame+1-aug_frame_shift*(aug_number-1)
        uniform_preds.append([round(start, 5), round(end,5)])
    
    iou_clip = 0.
    jacc_clip = 0.
    for i in range(num_segments):
        best_iou = 0.
        best_jacc = 0.
        for j in range(num_segments):
            clip_boundary = uniform_preds[j]
            current_iou = compute_iou(clip_boundary, label[i])
            if current_iou > best_iou:
                best_iou = current_iou
            current_jacc = compute_jacc(clip_boundary, label[i])
            if current_jacc > best_jacc:
                best_jacc = current_jacc
        iou_clip += best_iou
        jacc_clip += best_jacc
    ious.append(iou_clip/num_segments)
    jaccs.append(jacc_clip/num_segments)
print(round(100*np.nanmean(ious),3), round(100*np.nanmean(jaccs),3))

36.392 40.432


In [177]:
round(0.35794000734444126, 4)

0.3579

In [136]:
122.56/13

9.427692307692308

In [143]:
interval = 122.56/13
segs = []
for i in range(13):
    if i % 2 == 0:
        continue
    segs.append((i*interval, (i+1)*interval))

In [144]:
segs

[(9.427692307692308, 18.855384615384615),
 (28.283076923076923, 37.71076923076923),
 (47.13846153846154, 56.566153846153846),
 (65.99384615384615, 75.42153846153846),
 (84.84923076923077, 94.27692307692308),
 (103.70461538461538, 113.13230769230769)]

In [145]:
start

1366.4001305483027